# ПРЕБАРУВАЊЕ СО УНИФОРМНА ЦЕНА

## Тежински граф

In [23]:
class Graph():
    
    def __init__(self):
        self.dict = {}
        
    def add_vertex(self, vertex):
        if vertex not in self.dict:
            self.dict[vertex] = {}
        
    def add_edge(self, edge):
        # 'A': {'B': 3, 'C': 4}
        weight, vertex1, vertex2 = edge
        self.dict[vertex1][vertex2] = weight
        #ako e vo dvete nasokis
        self.dict[vertex2][vertex1] = weight
        
    def give_vertices(self): #go  pokazhuva rechnikot so kluchovite vo lista
        return list(self.dict.keys())
    
    def give_edges(self): #go pokazhuva rechnikot vo lista so torki so zavisnostite so cenata
        edges = []
        for vertex1 in self.dict:
            for vertex2 in self.dict[vertex1]:
                edges.append((self.dict[vertex1][vertex2], vertex1, vertex2))
        return edges
    
    def give_neighbors(self, vertex):
        neighbors = []
        for neighbor in self.dict[vertex]:
            neighbors.append((self.dict[vertex][neighbor], neighbor))
        return neighbors

In [24]:
g = Graph()
g.add_vertex('A')
g.add_vertex('B')
g.add_vertex('C')
g.add_vertex('D')
g.add_vertex('E')
g.add_vertex('F')
g.give_vertices()

['A', 'B', 'C', 'D', 'E', 'F']

In [25]:
g.add_edge((2, 'A', 'B'))
g.add_edge((4, 'A', 'C'))
g.add_edge((5, 'A', 'E'))
g.add_edge((1, 'B', 'E'))
g.add_edge((3, 'E', 'D'))
g.add_edge((2, 'E', 'F'))
g.add_edge((1, 'F', 'D'))
g.add_edge((3, 'C', 'D'))

g.give_edges()

[(2, 'A', 'B'),
 (4, 'A', 'C'),
 (5, 'A', 'E'),
 (2, 'B', 'A'),
 (1, 'B', 'E'),
 (4, 'C', 'A'),
 (3, 'C', 'D'),
 (3, 'D', 'E'),
 (1, 'D', 'F'),
 (3, 'D', 'C'),
 (5, 'E', 'A'),
 (1, 'E', 'B'),
 (3, 'E', 'D'),
 (2, 'E', 'F'),
 (2, 'F', 'E'),
 (1, 'F', 'D')]

In [26]:
g.give_neighbors('A')

[(2, 'B'), (4, 'C'), (5, 'E')]

In [21]:
import heapq

In [40]:
def uniform_cost_search(graph, initial_vertex, goal_vertex):
    
    if initial_vertex == goal_vertex:
        return
    
    queue = [(0, [initial_vertex])]
    heapq.heapify(queue)
    expanded = set()
    
    while queue:
        weight, list_to_expand = heapq.heappop(queue)
        vertex_to_expand = list_to_expand[-1]
        
        if vertex_to_expand == goal_vertex:
            return weight, list_to_expand
        
        for neighbor_weight, neighbor in graph.give_neighbors(vertex_to_expand):
            if neighbor not in expanded:
                heapq.heappush(queue, (weight + neighbor_weight, list_to_expand + [neighbor]))
                
        # ako neighbor e veke razgranet, togas sigurno sega razgranvanjeto ke e podolgo
        # (na pr SAG sekogas ke e poevtino od BGSAG posto SAG se sodrzi vo BGSAG, B i G
        # samo dodavat plus cena => to sigurno nema da e najevtino razgranvanje i nikogas
        # nema da ni pritreba, poshto sekogas najevtinoto se zema)
        
        expanded.add(vertex_to_expand)
                
# [(3, [A, C, E]), (4, [B, C, E]), ...]

In [41]:
uniform_cost_search(g, 'A', 'F')

(5, ['A', 'B', 'E', 'F'])

## Претурање вода

Имате на располагање 3 садови. Нивниот капацитет е 5, 8 и 10 соодветно.  На почетокот сите садови се празни. Задачата е да претурите 1 литар во садот од 5 литри. Можете да претурате вода меѓу садовите, но не знаете точно колку претурате бидејќи садовите не се обележани. Единствено може да го испразните садот од кој претурате или да го наполните до врв садот во кој претурате. Постои и чешма од која може да надополните некој сад до врвот. Секогаш може да ја истурите водата од некој сад. Скратено: имате на располагање 3 садови. Нивниот капацитет е (x, y, z) соодветно. Почетната состојба е (x_0, y_0, z_0). Задачата е да стигнете до посакуваната состојба.

Дополнително, секој нов наточен литар се казнува со 1 поен (еквивалентно на 1 потег), а секој истурен литар се казнува со 3 поени (еквивалетно на 3 потези).

In [6]:
CAPACITY = (5, 8, 10)

In [7]:
def transfer_water(state, source_index, sink_index):
    state_list = list(state)
    
    if CAPACITY[sink_index] - state[sink_index] >= state[source_index]:
        state_list[sink_index] = state[sink_index] + state[source_index]
        state_list[source_index] = 0
        return tuple(state_list)
    
    state_list[sink_index] = CAPACITY[sink_index]
    state_list[source_index] = state[source_index] - (CAPACITY[sink_index] - state[sink_index])
    return tuple(state_list)
    
    
# (4, (4, 5, 10))
# (4, (1, 5, 10))

In [8]:
transfer_water((4, 5, 10), 0, 1)

(1, 8, 10)

In [9]:
def expand_state(state):
    states_list = []
    
    for index, element in enumerate(state):
        if element > 0:
            new_state = list(state)
            new_state[index] = 0
            states_list.append((3 * state[index], tuple(new_state)))
            
    for index, element in enumerate(state):
        if element < CAPACITY[index]:
            new_state = list(state)
            new_state[index] = CAPACITY[index]
            states_list.append(((CAPACITY[index] - state[index]), tuple(new_state)))
            
    for index1, element1 in enumerate(state):
        for index2, element2 in enumerate(state):
            if index1 != index2:
                new_state = transfer_water(state, index1, index2)
                if new_state not in states_list and new_state != state:
                    states_list.append((0, new_state))
                    
    return states_list

In [10]:
expand_state((2, 0, 0))

[(6, (0, 0, 0)),
 (3, (5, 0, 0)),
 (8, (2, 8, 0)),
 (10, (2, 0, 10)),
 (0, (0, 2, 0)),
 (0, (0, 0, 2))]

In [11]:
import heapq

In [12]:
def uniform_cost_search(initial_state, goal_state):
    
    queue = [(0, [initial_state])]
    heapq.heapify(queue)
    expanded = set()
    
    while queue:
        weight, list_to_expand = heapq.heappop(queue)
        #print('\nMomentalnata tezina e {}'.format(weight))
        state_to_expand = list_to_expand[-1]
        
        if state_to_expand == goal_state:
            return (weight, list_to_expand)
        
        for next_weight, next_state in expand_state((state_to_expand)):
            if next_state not in expanded:
                heapq.heappush(queue, (weight + next_weight, list_to_expand + [next_state]))
                
        expanded.add(state_to_expand)
    

In [13]:
uniform_cost_search((0, 0, 0), (1, 0, 0))

(41,
 [(0, 0, 0),
  (0, 0, 10),
  (0, 8, 2),
  (2, 8, 0),
  (2, 0, 8),
  (0, 2, 8),
  (5, 2, 3),
  (0, 7, 3),
  (0, 8, 3),
  (0, 1, 10),
  (0, 1, 0),
  (0, 0, 1),
  (1, 0, 0)])

## Градови

In [6]:
class Graph():
    
    def __init__(self):
        self.dict = {}
        
    def add_vertex(self, vertex):
        if vertex not in self.dict:
            self.dict[vertex] = {}
        
    def add_edge(self, edge):
        # 'A': {'B': 3, 'C': 4}
        weight, vertex1, vertex2 = edge
        self.dict[vertex1][vertex2] = weight
        self.dict[vertex2][vertex1] = weight
        
    def give_vertices(self):
        return list(self.dict.keys())
    
    def give_edges(self):
        edges = []
        for vertex1 in self.dict:
            for vertex2 in self.dict[vertex1]:
                edges.append((self.dict[vertex1][vertex2], vertex1, vertex2))
        return edges
    
    def give_neighbors(self, vertex):
        return list(self.dict[vertex].items())

In [16]:
info = open('data/distances.txt').read().strip()
g = Graph()

for row in info.split('\n'):
    vertex1, vertex2, weight = row.split(' ')
    g.add_vertex(vertex1)
    g.add_vertex(vertex2)
    g.add_edge((int(weight), vertex1, vertex2))
    
g.give_edges()[:5]

[(39, 'KU', 'SK'),
 (61, 'KU', 'KP'),
 (51, 'KU', 'KR'),
 (39, 'SK', 'KU'),
 (44, 'SK', 'TE')]

In [17]:
g.give_neighbors('SK')

[('KU', 39), ('TE', 44), ('VE', 55)]

In [14]:
import heapq

def uniform_cost_search(graph, initial_vertex, goal_vertex):
    
    queue = [(0, [initial_vertex])]
    heapq.heapify(queue)
    expanded = set()
    
    while queue:
        weight, list_to_expand = heapq.heappop(queue)
        vertex_to_expand = list_to_expand[-1]
        
        if vertex_to_expand == goal_vertex:
            return weight, list_to_expand
        
        if vertex_to_expand not in expanded:
            for next_vertex, next_weight in graph.give_neighbors(vertex_to_expand):
                if next_vertex not in expanded:
                    heapq.heappush(queue, (weight+next_weight, list_to_expand + [next_vertex]))
            expanded.add(vertex_to_expand)

In [19]:
%%time
uniform_cost_search(g, 'GE', 'TE')

CPU times: total: 0 ns
Wall time: 0 ns


(204, ['GE', 'NG', 'VE', 'SK', 'TE'])

## Преминување преку мост

Adam, Bob, Clair and Dave are out walking: They come to rickety old wooden bridge. The bridge is weak and only able to carry the weight of two of them at a time. Because they are in a rush and the light is fading they must cross in the minimum time possible and must carry a torch (flashlight,) on each crossing.

They only have one torch and it can't be thrown. Because of their different fitness levels and some minor injuries they can all cross at different speeds. Adam can cross in 1 minute, Bob in 2 minutes, Clair in 5 minutes and Dave in 10 minutes.

Adam, the brains of the group thinks for a moment and declares that the crossing can be completed in 17 minutes. There is no trick. How is this done?

In [30]:
torch_index = 4
minutes = (1, 2, 5, 10)

In [51]:
def move(state, how_many_people, people_index):
    state_list = list(state)
    
    if how_many_people == 1:
        if state_list[torch_index] == 1:
            state_list[torch_index] = state_list[people_index] = 0
        else:
            state_list[torch_index] = state_list[people_index] = 1
        weight = minutes[people_index]
    
    if how_many_people == 2:
        if state_list[torch_index] == 1:
            state_list[torch_index] = state_list[people_index[0]] = state_list[people_index[1]] = 0
        else:
            state_list[torch_index] = state_list[people_index[0]] = state_list[people_index[1]] = 1
        weight = max(minutes[people_index[0]], minutes[people_index[1]])
            
    return weight, tuple(state_list)

In [57]:
state = (0, 1, 1, 0, 1)
move(state, 2, (0, 3))

(10, (0, 1, 1, 0, 0))

In [58]:
def expand_state(state):
    states = []
    for index1 in range(4):
        if state[index1] == state[torch_index]:
            states.append(move(state, 1, index1))
            
        for index2 in range(4):
            if index1 != index2 and state[index1] == state[index2] == state[torch_index]:
                if move(state, 2, (index1, index2)) not in states:    
                    states.append(move(state, 2, (index1, index2)))
    return states

In [59]:
expand_state(state)

[(2, (0, 0, 1, 0, 0)), (5, (0, 0, 0, 0, 0)), (5, (0, 1, 0, 0, 0))]

In [63]:
import heapq

def uniform_cost_search(initial_state, goal_state):
    queue = [(0, [initial_state])]
    heapq.heapify(queue)
    expanded = set()
    
    while queue:
        weight, list_to_expand = heapq.heappop(queue)
        state_to_expand = list_to_expand[-1]
        
        if state_to_expand == goal_state:
            return weight, list_to_expand
        
        if state_to_expand not in expanded:
            for next_weight, next_state in expand_state(state_to_expand):
                if next_state not in expanded:
                    heapq.heappush(queue, (next_weight+weight, list_to_expand + [next_state]))
            expanded.add(state_to_expand)

In [62]:
uniform_cost_search(
    initial_state = (0, 0, 0, 0, 0),
    goal_state = (1, 1, 1, 1, 1)
)

(17,
 [(0, 0, 0, 0, 0),
  (1, 1, 0, 0, 1),
  (0, 1, 0, 0, 0),
  (0, 1, 1, 1, 1),
  (0, 0, 1, 1, 0),
  (1, 1, 1, 1, 1)])

## Преминување преку мост - детално објаснато

Adam, Bob, Clair and Dave are out walking: They come to rickety old wooden bridge. The bridge is weak and only able to carry the weight of two of them at a time. Because they are in a rush and the light is fading they must cross in the minimum time possible and must carry a torch (flashlight,) on each crossing.

They only have one torch and it can't be thrown. Because of their different fitness levels and some minor injuries they can all cross at different speeds. Adam can cross in 1 minute, Bob in 2 minutes, Clair in 5 minutes and Dave in 10 minutes.

Adam, the brains of the group thinks for a moment and declares that the crossing can be completed in 17 minutes. There is no trick. How is this done?

In [11]:
cost = (1, 2, 5, 10) # цената е колку време му треба на секој човек да го помине мостот:
# на првиот човек му треба 1 минута, на вториот 2, на третиот 5, на четвртиот 10

In [12]:
# состојбата ја обележувам како торка од 5 елементи. Првите 4 елементи ја покажуваат
# позицијата на луѓето. Ако се наоѓаат пред мостот, се на позиција 0, ако го преминале мостот, се на позиција 1.
# Последниот елемент ја покажува позицијата на фаќелот (повторно е 0 или 1). 

In [13]:
def two_people_go(state, person_1_index, person_2_index):
    # фунцкија со која двајца се испраќаат до позиција 1
    
    state_list = list(state)
    state_list[person_1_index] = 1
    state_list[person_2_index] = 1
    state_list[4] = 1
    return (max(cost[person_1_index], cost[person_2_index]), tuple(state_list))
        # времето потребно за двајца да преминат преку мостот е еднакво на
        # времето потребно за да премине преку мостот поспориот од нив двајцата

In [14]:
two_people_go((0, 0, 0, 0, 0), 0, 1) # пример

(2, (1, 1, 0, 0, 1))

In [15]:
def two_people_come(state, person_1_index, person_2_index):
    # функција со која двајца се испраќаат до позиција 0

    state_list = list(state)
    state_list[person_1_index] = 0
    state_list[person_2_index] = 0
    state_list[4] = 0
    return (max(cost[person_1_index], cost[person_2_index]), tuple(state_list))

In [16]:
two_people_come((1, 1, 0, 0, 1), 0, 1) # пример

(2, (0, 0, 0, 0, 0))

In [17]:
def one_person_goes(state, person_index):
    # функција со која се испраќа еден човек до страна 1

    state_list = list(state)
    state_list[person_index] = 1
    state_list[4] = 1
    return (cost[person_index], tuple(state_list))

In [18]:
one_person_goes((0, 0, 0, 0, 0), 0) # пример

(1, (1, 0, 0, 0, 1))

In [19]:
def one_person_comes(state, person_index):
    # функција со која се испраќа еден човек до страна 0

    state_list = list(state)
    state_list[person_index] = 0
    state_list[4] = 0
    return (cost[person_index], tuple(state_list))

In [20]:
one_person_comes((1, 1, 0, 0, 1), 0) # пример

(1, (0, 1, 0, 0, 0))

In [21]:
def expand_state(state):
    states = []
    pos0, pos1, pos2, pos3, torch_pos = state
    if torch_pos == 0: # ако фаќелот е на позиција 0, тогаш треба да пренесуваме
    # луѓе кои се наоѓаат исто така на позиција 0, затоа што кај нив се наоѓа фаќелот
        for i in range(4): #broi od 0 do 3
            if state[i] == 0: # човекот може да оди кон позиција 1, само
            # ако моментално се наоѓа во позиција 0
                states.append(one_person_goes(state, i))

        for i in range(4):
            for j in range(4):
                if i != j and state[i] == 0 and state[j] == 0:
                # и двајцата треба да се наоѓаат во позиција 0, притоа тие мора да се
                # двајца различни луѓе, за оваа функција точно да работи
                    if two_people_go(state, i, j) not in states:
                        states.append(two_people_go(state, i, j))
                            
    if torch_pos == 1:
        for i in range(4):
            if state[i] == 1:
                states.append(one_person_comes(state, i))
        for i in range(4):
            for j in range(4):
                if i != j and state[i] == 1 and state[j] == 1:
                    if two_people_come(state, i, j) not in states:
                        states.append(two_people_come(state, i, j))
                            
    return states

In [22]:
expand_state((0, 0, 0, 0, 0)) # пример

[(1, (1, 0, 0, 0, 1)),
 (2, (0, 1, 0, 0, 1)),
 (5, (0, 0, 1, 0, 1)),
 (10, (0, 0, 0, 1, 1)),
 (2, (1, 1, 0, 0, 1)),
 (5, (1, 0, 1, 0, 1)),
 (10, (1, 0, 0, 1, 1)),
 (5, (0, 1, 1, 0, 1)),
 (10, (0, 1, 0, 1, 1)),
 (10, (0, 0, 1, 1, 1))]

In [23]:
import heapq

In [24]:
def uniform_cost_search(initial_state, goal_state):
    if initial_state == goal_state:
        return
    
    expanded = set()
    queue = [(0, [initial_state])]
    heapq.heapify(queue)
    
    while queue:
        weight, list_to_expand = heapq.heappop(queue)
        state_to_expand = list_to_expand[-1]
        
        if state_to_expand == goal_state:
            return (weight, list_to_expand)
        
        if state_to_expand in expanded:
            continue
        
        
            
        for new_weight, new_state in expand_state(state_to_expand):
            if new_state not in expanded:
                heapq.heappush(queue, (weight + new_weight, list_to_expand + [new_state]))
                
        

In [25]:
uniform_cost_search((0, 0, 0, 0, 0), (1, 1, 1, 1, 1))

(17,
 [(0, 0, 0, 0, 0),
  (1, 1, 0, 0, 1),
  (0, 1, 0, 0, 0),
  (0, 1, 1, 1, 1),
  (0, 0, 1, 1, 0),
  (1, 1, 1, 1, 1)])